In [2]:
import re
import numpy as np
import math 

In [6]:
def expression_to_matrix(exp):
    no_coeffs = re.sub(r'x_\d+', '', exp)
    tokens = re.findall(r'[+-]?\d+', no_coeffs)
    numbers = [int(token) for token in tokens]
    matrix = np.array(numbers)

    return matrix

def parse_lp_problem(file_path):
    with open(file_path, 'r') as file:
        lines = file.read().split('\n')

    obj = expression_to_matrix(lines[1])
    if lines[0] == 'min':
        obj = -obj
    vars = len(obj)
    add_vars = 0
    right_side = []

    def restriction_to_function(restr):
        splitted = restr.split(' ')
        right_side.append(float(splitted[-1]))
        restr = expression_to_matrix(' '.join(splitted[:-2]))
        for i in range(add_vars):
            restr = np.append(restr, 0)
        return restr

    all_restr = []
    for i in range(len(lines) - 2):
        restr = lines[i+2]
        if '>=' in restr:
            restr = restriction_to_function(restr)
            restr = np.append(restr, -1)
            add_vars += 1
        elif '<=' in restr:
            restr = restriction_to_function(restr)
            restr = np.append(restr, +1)
            add_vars += 1
        else:
            restr = restriction_to_function(restr)
        all_restr.append(restr)

    full_matrix = np.append(all_restr[0], np.zeros(vars + add_vars - len(all_restr[0])))
    for i in range(len(all_restr) - 1):
        with_zeroes = np.append(all_restr[i+1], np.zeros(vars + add_vars - len(all_restr[i+1])))
        full_matrix = np.vstack((full_matrix, with_zeroes))

    obj = np.append(obj, np.zeros(add_vars))

    return obj.tolist(), full_matrix.tolist(), right_side, vars

file_path = 'example.txt'
c, A, b, vars = parse_lp_problem(file_path)

In [4]:
def to_tab(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]

def optimization_check(tab):
    return any(x > 0 for x in tab[-1][:-1])

def get_position(tab):
    z = tab[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)

    restrictions = []
    for eq in tab[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    if (all([r == math.inf for r in restrictions])):
        raise Exception("Linear program is unbounded. ")

    row = restrictions.index(min(restrictions))
    return row, column

def pivot_step(tab, pivot_position):
    new_tableau = [[] for eq in tab]

    i, j = pivot_position
    pivot_value = tab[i][j]
    new_tableau[i] = np.array(tab[i]) / pivot_value

    for eq_i, eq in enumerate(tab):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tab[eq_i][j]
            new_tableau[eq_i] = np.array(tab[eq_i]) - multiplier

    return new_tableau

def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tab):
    columns = np.array(tab).T
    solutions = []
    for column in columns[:-1]:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)

    return solutions

def simplex_method(c, A, b):
    tab = to_tab(c, A, b)
    while optimization_check(tab):
        pivot_position = get_position(tab)
        tab = pivot_step(tab, pivot_position)

    return get_solution(tab)

In [5]:
solution = simplex_method(c, A, b)
optimal_value = np.dot(solution, c)
print('solution: ', solution[:vars])
print('optimal_value: ', optimal_value) 

solution:  [9200.0, 500.0, 300.0, 0]
optimal_value:  9700.0
